## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [180]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        if len(codigo) == 4 and codigo.isdigit():
            self.codigo = codigo
        else:
            raise ValueError("El código debe ser una cadena de 4 dígitos.")
        
        if 1 <= len(nombre) <= 100:
            self.nombre = nombre
        else:
            raise ValueError("El nombre debe tener entre 1 y 100 caracteres.")
        
        self._precio = None
        self._cantidad = None
        self.precio = precio
        self.tipo = tipo if 0 <= len(tipo) <= 20 else ''
        self.cantidad = cantidad

    def valorTotal(self):
        return self.precio * self.cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor
        else:
            pass

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 100:
            self._cantidad = valor
        else:
            pass


In [181]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [182]:
class Oferta:
    def __init__(self, codigos=None, tipos=None):
        self.codigos = codigos if codigos else []
        self.tipos = tipos if tipos else []

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos


class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=None, tipos=None):
        super().__init__(codigos, tipos)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return (producto.precio * cantidad) * (self.descuento / 100)
        return 0


class Oferta2x1(Oferta):
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return (cantidad // 2) * producto.precio
        return 0

In [183]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [184]:
class Catalogo:
    def __init__(self):
        self.productos = {}  
        self.ofertas = []    

    def agregar(self, *productos):
        for producto in productos:
            self.productos[producto.codigo] = producto

    def buscar(self, codigo):
        return self.productos.get(codigo)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        descuentos = [oferta.calcularDescuento(producto, cantidad) for oferta in self.ofertas if oferta.esAplicable(producto)]
        return max(descuentos, default=0)  

    def vender(self, producto, cantidad):
        if producto.codigo in self.productos:
            p = self.productos[producto.codigo]
            if p.cantidad >= cantidad:
                p.cantidad -= cantidad 
            else:
                raise ValueError(f"No hay suficiente stock para el producto {producto.nombre}.")
        else:
            raise ValueError(f"Producto {producto.nombre} no encontrado en el catálogo.")

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos.values())

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos.values())

    def guardar(self, archivo):
        with open(archivo, 'w') as f:
            f.write("codigo,nombre,precio,tipo,cantidad\n")
            for producto in self.productos.values():
                f.write(f"{producto.codigo},{producto.nombre},{producto.precio},{producto.tipo},{producto.cantidad}\n")

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, 'r') as f:
            next(f)
            for linea in f:
                codigo, nombre, precio, tipo, cantidad = linea.strip().split(',')
                producto = Producto(codigo, nombre, int(precio), tipo, int(cantidad))
                catalogo.agregar(producto)
        return catalogo

    def informe(self):
        tipos = {producto.tipo for producto in self.productos.values()}
        informe = (
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: {self.valorTotal / self.cantidadUnidades:.2f}\n"
            f"Valor total: {self.valorTotal:.2f}\n"
            f"Tipos de productos: {', '.join(tipos)}\n"
            "Ofertas:\n"
        )
        for oferta in self.ofertas:
            tipo_oferta = "Oferta 2x1" if isinstance(oferta, Oferta2x1) else f"Descuento del {oferta.descuento}%"
            informe += f"{tipo_oferta}\n"
        return informe


In [185]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [186]:
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre  
        self.cuit = cuit  

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if not value or not (1 <= len(value) <= 100):
            return
        self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        import re
        pattern = r'^\d{2}-\d{8}-\d{1}$' 
        if not re.match(pattern, value):
           
            return
        self._cuit = value



In [187]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [188]:
class Factura:
    ultimaFactura_num = 0  

    @classmethod
    def ultimaFactura(cls, numero):
        cls.ultimaFactura_num = numero

    @classmethod
    def nuevoNumero(cls):
        cls.ultimaFactura_num += 1
        return cls.ultimaFactura_num

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.productos = []
        self.cantidades = []
        self.descuentos_aplicados = [] 

    def agregar(self, producto, cantidad):
        if producto.codigo in [p.codigo for p in self.productos]:
            indice = [p.codigo for p in self.productos].index(producto.codigo)
            cantidad_adicional = cantidad
            self.cantidades[indice] += cantidad_adicional
            self.catalogo.vender(producto, cantidad_adicional) 
        else:
            self.productos.append(producto)
            self.cantidades.append(cantidad)
            self.catalogo.vender(producto, cantidad)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(self.cantidades)

    @property
    def subtotal(self):
        return sum(producto.precio * cantidad for producto, cantidad in zip(self.productos, self.cantidades))

    @property
    def descuentos(self):
        total_descuento = 0
        self.descuentos_aplicados = []  
        for producto, cantidad in zip(self.productos, self.cantidades):
            ofertas_aplicables = [oferta for oferta in self.catalogo.ofertas if oferta.esAplicable(producto)]
            descuentos = [oferta.calcularDescuento(producto, cantidad) for oferta in ofertas_aplicables]
            if descuentos:
                max_descuento = max(descuentos)
                total_descuento += max_descuento
                for oferta in ofertas_aplicables:
                    descuento = oferta.calcularDescuento(producto, cantidad)
                    if descuento == max_descuento:
                        if isinstance(oferta, OfertaDescuento):
                            descripcion = f"Descuento {oferta.descuento}% en {producto.nombre}"
                        elif isinstance(oferta, Oferta2x1):
                            descripcion = f"Oferta 2x1 en {producto.nombre}"
                        else:
                            descripcion = "Otra oferta"
                        self.descuentos_aplicados.append(descripcion)
                        break  
        return total_descuento

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        salida = f"Cliente: {self.cliente.nombre}\n"
        for producto, cantidad in zip(self.productos, self.cantidades):
            salida += f"{cantidad} {producto.nombre}\n"
        salida += f"SUBTOTAL: {self.subtotal:.2f}\n"
        
        for descripcion in self.descuentos_aplicados:
            salida += f"{descripcion}\n"
        
        salida += f"DESCUENTO: {self.descuentos:.2f}\n"
        salida += f"TOTAL: {self.total:.2f}\n"
        return salida


In [189]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [190]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00
assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
